Processing video data often takes quite some time, especially when you have many videos. In this notebook we show some examples of how you could apply movement detection to multiple videos and how to optimize processing speed.

In [ ]:
from pathlib import Path

import birdwatcher as bw
import birdwatcher.movementdetection as md

### Select videos and settings

First, specify a directory with all videos you would like to do movement detection on. Make sure the filename extension is correct.

In [ ]:
videodir = '../videos'

# create a list of videofilepaths
videofilepaths = list(Path(videodir).glob('*.mp4'))
videofilepaths

Then, load each video as VideoFileStream object into a list:

In [ ]:
vfs_list = [bw.VideoFileStream(path) for path in videofilepaths]
vfs_list

Specify the settings you want to use for movement detection. Note that these settings will be applied to all videos in the directory!

In [ ]:
# optional: if you want to do movement detection only on part of the videos
startat = '00:00:00'   # in HOURS:MM:SS
nframes = None

# specify h1, h2, w1, w2, or choose None to use the whole frames
roi = None   # region of interest
nroi = None   # nót region of interest

In [ ]:
settings = {'bgs_params':  {'History': 3,
                            'ComplexityReductionThreshold': 0.05,
                            'BackgroundRatio': 0.1,
                            'NMixtures': 7,
                            'VarInit': 15,
                            'VarMin': 10,
                            'VarMax': 75,
                            'VarThreshold': 70,
                            'VarThresholdGen': 9,
                            'DetectShadows': False,
                            'ShadowThreshold': 0.5,
                            'ShadowValue': 0},

            'processing':  {'color': False,   # booleans only
                            'resizebyfactor': 1,   # use '1' for no change in size
                            'blur': 10,   # use '0' for no blur
                            'morphologyex': True}}   # booleans only

### Batch detect movement - normal

Do movement detection on multiple videos, in which the same parameters values will be applied to each video. From each video a coordinate array and (optionally) movementvideo will be saved. 

Run the batch function, without saving the coordinate array as compressed archived folder:

In [ ]:
%%time
md.batch_detect_movement(vfs_list, settings, startat, nframes, roi, nroi, 
                         bgs_type=bw.BackgroundSubtractorMOG2, analysispath='output/not_archived/', 
                         ignore_firstnframes=50, overwrite=True, resultvideo=False, 
                         archived=False, nprocesses=8)

The movement data are saved as darr coordinate arrays in the specified directory. You can load and look at the data similarly as shown in notebook 5.

### Batch detect movement - archived coords

You can also run batch detect movement in which the coordinates arrays are saved as compressed archived folders (.tar.xz) which might be faster and saves disk memory. This is especially useful when you have a lot of movement data.

Run the batch function, `archived` set as `True`:

In [ ]:
%%time
md.batch_detect_movement(vfs_list, settings, startat, nframes, roi, nroi, 
                         bgs_type=bw.BackgroundSubtractorMOG2, analysispath='output/archived/', 
                         ignore_firstnframes=50, overwrite=True, resultvideo=False,
                         archived=True, nprocesses=8)

### Open archived coordinates array

When you've used batch detect movement and saved the coordinate arrays as archived data folders , you'll need to unpack the compressed coordinate array first before you can look at the data. 

#### Using a context manager

One way to do that is by temporarily decompressing the coordinate data using a context manager, do some stuff with the data, and at the end of the cell, the decompressed coordinate data is removed again.

In [ ]:
movementpath = 'output/archived/movement_zf20s_low'

with bw.open_archivedcoordinatedata(f'{movementpath}/coords.darr.tar.xz') as coords:
    # Here you can do whether you want with the data, such as:
    coordscount = coords.get_coordcount()
    coordsmean = coords.get_coordmean()

In [ ]:
type(coordscount)

Now you can look at coordscount and coordsmean, but you don't have access to the raw coordinate array anymore. Using this methods limits memory usage.

#### Decompress archived coords

Another option is to decompress the archived darr array to be able to access the coordinate data normally.

In [ ]:
movementpath = 'output/archived/movement_zf20s_low'

coords = bw.extract_archivedcoordinatedata(f'{movementpath}/coords.darr.tar.xz')

Now, the archived coordinate array is decompressed and you can access it normally:

In [ ]:
coords.show(framerate=25)

When you're at the end of the notebook, you could delete the coordinates array again to save disk memory. The archived coordinate array will still be there, which you can always open again.

In [ ]:
bw.delete_coordinatearray(f'{movementpath}/coords.darr')